In [81]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import re
import statistics
import fastai
from keras.preprocessing.text import Tokenizer
import fastai.text.learner as learner
import fastai.text.models as models
from fastai.text.models import AWD_LSTM
from fastai.text.models import awd_lstm_lm_config

#from fastai.callbacks.lr_finder import lr_find
from fastai import *
from fastai.vision import *
import string
from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")
nltk.download('punkt')
sum_noise = list(string.punctuation)

from nltk.tokenize import word_tokenize

#import fastai
#import fastai.text.data
from sklearn.model_selection import train_test_split
path = ''

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ImportError: cannot import name 'lr_find' from 'fastai.text' (/home/victor/anaconda3/lib/python3.7/site-packages/fastai/text/__init__.py)

In [89]:
from fastai.callbacks import LRFinder

ModuleNotFoundError: No module named 'fastai.callbacks'

In [28]:
def token_text(text):
    return [word for word in word_tokenize(text.lower()) if word not in sum_noise]

In [11]:
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

# Формируем сбалансированный датасет
sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['text'].values[:sample_size],
                           data_negative['text'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

In [22]:
def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    text = text.lower().replace("USER", "")
    text = text.lower().replace("rt", "")
    text = text.lower().replace("URL", "")
    text = text.lower().replace("", "")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()


data = [preprocess_text(t) for t in raw_data]
data_tok = pd.Series(data).apply(token_text)

In [38]:
WORDS = set()
for sent in data_tok:
    for w in sent:
        WORDS.add(w)

In [14]:
df_train=pd.DataFrame(columns=['Text', 'Label'])
df_test=pd.DataFrame(columns=['Text', 'Label'])

df_train['Text'], df_test['Text'], df_train['Label'], df_test['Label'] = train_test_split(data, labels, test_size=0.2, random_state=1)

In [15]:
df_val=pd.DataFrame(columns=['Text', 'Label'])

df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=1)

In [16]:
tokenizer=Tokenizer()
data_lm = fastai.text.data.TextDataLoaders.from_df(df_train, path=path, bs=16)

/home/victor/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [17]:
data_lm.show_batch()

,text,category
0,xxbos милаш xxrep 3 к xxrep 4 а любл xxrep 4 ю xxrep 4 т xxrep 4 е xxrep 3 б xxrep 5 я xxrep 3 с xxrep 3 и xxrep 3 л xxrep 3 ь xxrep 3 н xxrep 8 о xxrep 4 о xxrep 3 ч xxrep 3 е xxrep 4 н xxrep 6 ь круто тоб xxunk xxup url,1
1,xxbos бала xxunk м xxunk xxunk а май ст не варенье жа ып xxunk нб з аз р сол xxunk ша ым ес ме т с п отыр аны xxunk са xxunk,1
2,xxbos xxup user ну может он и клевый но на самом бу моменте мой комп видимо xxunk обо мне просто взял и вырубился так что я гли пойду смотреть,1
3,xxbos хотя я знаю что мы ни раз с тобой встретимся и все могло быть иначе если бы ты не xxunk в самом начале тебе было не нужно теперь,1
4,xxbos я до сих пор вспоминаю наш последний диалог с xxup user и понимаю что если бы не алина xxunk и другие то ты была бы сейчас с нами,0
5,xxbos xxup user ты с чего взяла что про дуру я тебе написала не знаешь а пишешь не надо обзываться алин ты же вроде взрослый и умный человек,1
6,xxbos xxup user и кстати сначала мне тем более так не показалось он не показал и виду что у него кто то есть так что я чиста 3,0
7,xxbos xxup user да ладно тебе не xxunk я же пошутил ахахаха d ладно я пошутил что пошутил шутка в шутке про шутку короче ты понял xxup user,1
8,xxbos xxup user xxup user на меня ма наорала что под ее кроватью пыль не так лежит как лежала до их отъезда а ты мне тут про xxunk,1


In [67]:
weights_pretrained = '/models/lm_5_ep_lr2-3_5_stlr'
itos_pretrained = '/models/itos'
pretained_data = (weights_pretrained, itos_pretrained)

In [68]:
config = awd_lstm_lm_config.copy()
config['n_hid'] = 1150
learn_lm = learner.language_model_learner(data_lm, AWD_LSTM, config=config, pretrained=False, pretrained_fnames=pretained_data, drop_mult=0.3)
learn_lm.freeze()

In [74]:
learn_lm.lr_find()
learn_lm.recorder.plot()

AttributeError: 'LMLearner' object has no attribute 'lr_find'